# Restricted Facility Location

### `SMC` approach

In [1]:
# import libraries
from scipy.special import softmax

## basic imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings('ignore')
from IPython.display import clear_output
import seaborn as sns
import cvxpy as cp
import time
clear_output()
myseed = '2404' 
np.random.seed(int(myseed))

In [2]:
## problem dimensions

B = int(10) 
LAMBDA_extra = 10


TITLE = 'at.csv' 

R = 2e-1
REL_PARAM = 1e-12

### utils

In [3]:
def proj_simplex_vec(V, z=1):
    n_features = V.shape[1]
    U = np.sort(V, axis=1)[:, ::-1]
    z = np.ones(len(V)) * z
    cssv = np.cumsum(U, axis=1) - z[:, np.newaxis]
    ind = np.arange(n_features) + 1
    cond = U - cssv / ind > 0
    rho = np.count_nonzero(cond, axis=1)
    theta = cssv[np.arange(len(V)), rho - 1] / rho
    return np.maximum(V - theta[:, np.newaxis], 0)

# from https://github.com/cvxgrp/dccp/blob/master/dccp/linearize.py

def linearize(expr):
    """Returns the tangent approximation to the expression.

    Gives an elementwise lower (upper) bound for convex (concave)
    expressions. No guarantees for non-DCP expressions.

    Args:
        expr: An expression.

    Returns:
        An affine expression.
    """
    if expr.is_affine():
        return expr
    else:
        if expr.value is None:
            raise ValueError(
                "Cannot linearize non-affine expression with missing variable values."
            )
        tangent = np.real(expr.value) #+ np.imag(expr.value)
        grad_map = expr.grad
        for var in expr.variables():
            if grad_map[var] is None:
                return None
            complex_flag = False
            if var.is_complex() or np.any(np.iscomplex(grad_map[var])):
                complex_flag = True
            if var.ndim > 1:
                temp = cp.reshape(
                    cp.vec(var - var.value), (var.shape[0] * var.shape[1], 1)
                )
                if complex_flag:
                    flattened = np.transpose(np.real(grad_map[var])) @ cp.real(temp) + \
                    np.transpose(np.imag(grad_map[var])) @ cp.imag(temp)
                else:
                    flattened = np.transpose(np.real(grad_map[var])) @ temp
                tangent = tangent + cp.reshape(flattened, expr.shape)
            elif var.size > 1:
                if complex_flag:
                    tangent = tangent + np.transpose(np.real(grad_map[var])) @ (cp.real(var) - np.real(var.value)) \
                    + np.transpose(np.imag(grad_map[var])) @ (cp.imag(var) - np.imag(var.value))
                else:
                    tangent = tangent + np.transpose(np.real(grad_map[var])) @ (var - var.value)
            else:
                if complex_flag:
                    tangent = tangent + np.real(grad_map[var]) * (cp.real(var) - np.real(var.value)) \
                    + np.imag(grad_map[var]) * (cp.imag(var) - np.imag(var.value))
                else:
                    tangent = tangent + np.real(grad_map[var]) * (var - var.value) 
        return tangent

### true data generation

In [4]:
def preprocess(fname,col_locations,col_weight='population',col_city='city'):
    
    ## loading part 
    try:
        
        raw_df = pd.read_csv(fname)
        raw_df.dropna(inplace=True,axis=1)
        N = raw_df.shape[0]

        if isinstance(col_weight,str):
            weights = raw_df[col_weight].values
        elif isinstance(col_weight,numbers.Number):
            weights = raw_df.iloc[:,int(col_weight)].values
        else:
            print('col_weight should either be a column name (str) or location (int)')
            weights = np.ones(N)
            
        weights = np.maximum(weights,0.0)
        if sum(weights)>0:
            weights /= np.sum(weights)
            
        if isinstance(col_city,str):
            city_names = raw_df[col_city].values
        elif isinstance(col_city,numbers.Number):
            city_names = raw_df.iloc[:,int(col_city)].values
        else:
            print('col_city should either be a column name (str) or location (int)')
            city_names = ['city #'+str(elem+1) for elem in range(N)]
            
        assert isinstance(col_locations,list),'col_locations must be a list of names (str) or indices (int)'
        locations = np.zeros((N,len(col_locations)))
        for id_loc,col_loc in enumerate(col_locations):
            if isinstance(col_loc,str):
                buf = raw_df[col_loc].values
            elif isinstance(col_loc,numbers.Number):
                buf = raw_df.iloc[:,int(col_loc)].values
            else:
                print('erroneous prescribed marker for city location... 0 value set for the linked dim.')
                buf = np.zeros(N)
            locations[:,id_loc] = buf.copy()

    except:
        print('-> error while loading your file')
        return None,None,None
    
    return weights,locations,list(city_names)

In [5]:
weights,locations,city_names = preprocess(TITLE,col_locations=['lat','lng'],col_weight='population',col_city='city')

In [6]:
N_train = min(len(locations),int(750))
id_train = np.random.choice(np.arange(len(locations)),replace=False,size=N_train)

omegas_train = locations[id_train]
weights_train = weights[id_train]
w_ext = omegas_train.shape[1]

### TRAINING

`parametric LP` 

In [7]:
# symmetry breakers tolerances
margin = .0
mu_DCA = 1e-8

# variables
radius = cp.Variable(1)
centrum = cp.Variable(w_ext)
mat_cvx = cp.Variable((B,w_ext))

# parameters
param = cp.Parameter((N_train,B),nonneg=True)

# basic feasible set 
LBS,UBS = np.min(omegas_train,0),np.max(omegas_train,0)
LBS *= (1-np.sign(LBS)*.02)
UBS *= (1+np.sign(UBS)*.02)
r_ref = abs(1/4*np.mean(UBS-LBS))
X = [mat_cvx[e]<=UBS for e in range(B)]+[mat_cvx[e]>=LBS for e in range(B)] 

# prior-knowledge encoding
symmetry_breaks = []
if B>1:
    for e in range(B-1):
        symmetry_breaks += [cp.sum(mat_cvx[e])+margin<=cp.sum(mat_cvx[e+1])]
        
h_bar_smc = LAMBDA_extra*cp.maximum(0,radius[0]-r_ref) if LAMBDA_extra>0 else 0.0
F_param_smc = h_bar_smc 
f1_dc = h_bar_smc+mu_DCA/2*(cp.sum_squares(mat_cvx)+cp.sum_squares(centrum)+cp.sum_squares(radius))

# objective function implementation + centrum "close enough" constraint
for e in range(B):
    X += [cp.norm(mat_cvx[e]-centrum,'inf')<=radius[0]]
    f1_dc += weights_train@cp.multiply(np.ones(N_train),cp.sum(cp.abs(np.ones((N_train,1))@cp.reshape(mat_cvx[e],(1,w_ext))-omegas_train),1))
    F_param_smc += weights_train@cp.multiply(param[:,e].T,cp.sum(cp.abs(np.ones((N_train,1))@cp.reshape(mat_cvx[e],(1,w_ext))-omegas_train),1))

prob_param_smc = cp.Problem(cp.Minimize(F_param_smc),X+symmetry_breaks)

In [8]:
'''-> h components evaluation <-'''
def heval(mat_val,data=omegas_train,weights=weights_train):
    buf = np.zeros((len(data),B))
    for e in range(B):
        buf[:,e] = np.sum(np.abs(np.outer(np.ones(N_train),mat_val[e])-data),1)
    return np.outer(weights,np.ones(B))*buf

'''-> ERM evaluation<-'''
def Feval(radius,center,mat_val,data=omegas_train,weights=weights_train):
    values = heval(mat_val,data,weights)
    return LAMBDA_extra*max(0,radius[0]-r_ref) + np.sum(np.min(values,1))

''' -> active components <- '''
def rho_active_sets(vals,rho=1e-10):
    indices = []
    minvals,maxvals = np.min(vals,1),np.max(vals,1)
    shifted_vals = (np.outer(maxvals,np.ones(vals.shape[1]))-np.array(vals))/(np.outer(maxvals,np.ones(vals.shape[1]))-np.outer(minvals,np.ones(vals.shape[1])))
    for idrow,row in enumerate(shifted_vals):
        tokeep = len(np.where(row>=1-rho)[0])
        indices.append(list(np.argsort(vals[idrow]))[:tokeep])
    return indices

In [9]:
'''-> computation of parameters <-
weights ndarray of shape (N_train,BB)
'''
def w2p(weights):
    param.value = weights
    
'''-> computation of f2_dc <-
weights ndarray of shape (N_train,BB) (opt)
'''
def w2l(weights):
    expr = 0.0
    loc_param = 1-weights
    for e in range(B):
        expr += weights_train@cp.multiply(loc_param[:,e].T,cp.sum(cp.abs(np.ones((N_train,1))@cp.reshape(mat_cvx[e],(1,w_ext))-omegas_train),1))
    return linearize(expr+mu_DCA/2*(cp.sum_squares(mat_cvx)+cp.sum_squares(centrum)+cp.sum_squares(radius)))
    
    
''' weights computations '''

def Q_opt(vals):
    buf = np.zeros(vals.shape)
    for s,sigma_s in enumerate(np.argmin(vals,1)):
        buf[s,sigma_s] += 1
    return buf

def Q_safe(vals,Q_current,Q_hat,Q_star,C):
    Q_next = np.zeros(Q_star.shape)
    combinations = np.zeros(len(Q_current))
    for _,Q_hat_elem in enumerate(Q_hat):
        num = np.sum((Q_current[_]-Q_star[_])*vals[_])
        denum = np.sum((Q_hat_elem-Q_star[_])*vals[_])
        if C*num>=denum:
            combinations[_] = 1
        else:
            combinations[_] = min(1,max(0,C*(num/max(1e-9,denum))))
        Q_next[_] = combinations[_]*Q_hat_elem+(1-combinations[_])*Q_star[_]
    return Q_next,combinations

def Q_hat_bb(vals,Q,param=1):
    minids = np.argmin(vals,1)
    Q_hat = Q.copy()
    for s,sigma_s in enumerate(minids):
        Q_hat[s] -= param*np.ones(vals.shape[1])
        Q_hat[s,sigma_s] += 2*param
    return proj_simplex_vec(Q_hat)

def Q_hat_mm(vals,param=1):
    minvals,maxvals = np.min(vals,1),np.max(vals,1)
    return proj_simplex_vec(param*(np.outer(maxvals,np.ones(vals.shape[1]))-np.array(vals))/(np.outer(maxvals,np.ones(vals.shape[1]))-np.outer(minvals,np.ones(vals.shape[1]))))

def Q_hat_softmin(vals,param=1,num_pert_inf=5e-7):
    pert_vals = vals+np.random.uniform(-num_pert_inf,num_pert_inf,vals.shape)
    return softmax(-param*pert_vals/np.maximum(1e-4,np.abs(np.outer(np.mean(vals,1),np.ones(B)))),axis=1)

For any $(l,l_+) \in [n_s]^2$,

$$M^{(s)}_{l_+,l} = \max_{u \,\in\, \mathcal{S}}\, h^{(s)}_{l_+}(u)-h^{(s)}_{l}(u) \quad \text{s.t.} \quad h^{(s)}_{l}(u)\leq h^{(s)}(u)$$


<b>FL</b>

* ...

<i>NOTE</i>: 

* if there exists $l_+ \in [n_s]$ such that $\hat{M}^{(s)}_{l_+,l}<0$ then $l$ is <b>not</b> active at any minimizer of $F$ over $\mathcal{S}$ and one sets $M^{(s)}_{l_+,l}=0$
* else, one sets $M^{(s)}_{l_+,l}=\hat{M}^{(s)}_{l_+,l}$

In [10]:
def bigM_FL(OMEGAS,weights=weights_train,Bparam=B,R_inf=R,c_centers=None):
    
    print('-> computation with R_inf = '+str(R_inf)+' ; p-norm with p = '+str(np.inf))
    print(' ')
    
    # init 
    N_loc = len(OMEGAS)
    data_loc = OMEGAS.copy()
    if c_centers is None:
        c_centers = np.zeros((Bparam,data_loc.shape[1]))
    M_list = [np.zeros((Bparam,Bparam)) for _ in range(N_loc)]

    # main loop
    for s,omega in enumerate(data_loc):
        for l_plus in range(Bparam):
            x_l_plus = c_centers[l_plus]
            x_l_plus_up = x_l_plus - R_inf*np.sign(omega-x_l_plus)
            up = np.sum(np.abs(omega-x_l_plus_up))
            for l in range(Bparam):
                if l==l_plus:
                    pass
                else:
                    x_l = c_centers[l]
                    x_l_down = x_l + np.minimum(R_inf,np.abs(omega-x_l))*np.sign(omega-x_l)
                    down = np.sum(np.abs(omega-x_l_down))
                    # computations
                    M_list[s][l_plus,l] = weights[s]*(up-down)
        if (s+1)%(np.ceil(N_loc/10))==0:
            print('ok for '+str((s+1)/N_loc*100)+'%')
    return M_list

In [11]:
NREPS = int(5)
Q_init_mats = []
for rep in range(NREPS):
    Q_init = np.random.uniform(0,1,(N_train,B))
    Q_init = np.array([q/np.sum(q) for q in Q_init])
    Q_init_mats.append(Q_init)

N_outer_max = int(10)
N_max = int(400)
tol_loc,tol_glob = 1e-8,5e-7

import gurobipy

local_env = gurobipy.Env()
local_env.setParam('TimeLimit', 180) # in seconds
local_env.setParam('Heuristics',.25) # in %

Academic license - for non-commercial use only - expires 2024-09-05
Using license file /Users/guiom_vds/gurobi.lic
Changed value of parameter TimeLimit to 180.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Heuristics to 0.25
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05


In [12]:
def RRAM(mode='am'):

    ## init
    fval = []
    t = []
    repIDs = []
    locopt = []
    endval_single_run = []
    endval = [] 

    if mode=='maxmin':
        param_schedule = lambda k: k**(2/3) 
        C_schedule = lambda k: 2/(k**(1/2)+3)
    elif mode=='softmin':
        param_schedule = lambda k: (3/2)**(k**(3/4)) if (3/2)**(k**(3/4))<1e16 else np.inf
        C_schedule = lambda k: 2/(k**(1/2)+3)
    elif mode=='baratt-boyd':
        param_schedule = lambda k: 1e-1
        C_schedule = lambda k: 1
    else: # am
        param_schedule = lambda k: 1
        C_schedule = lambda k: 0 

    print('=> '+str(mode)+' (START)')
    print(' ')

    for rep in range(NREPS):
        print('new rep #'+str(rep+1))
        print(' ')

        t_init = time.time()
        f_overallbest = np.inf
        mat_out = None
        rad_out = None
        center_out = None
        LO = False

        for k_outer in range(N_outer_max):

            print('OUTER #'+str(k_outer+1))
            print(' ')

            if k_outer==0:
                Q = Q_init_mats[rep].copy()
                w2p(Q)
                ref_val = np.inf
            else:
                components_ = heval(mat_cvx.value)
                Q = Q_opt(components_)
                w2p(Q)
                ref_val = Feval(radius.value,centrum.value,mat_cvx.value)
                print('restart w/ F = '+str(ref_val))
                print(' ')

            ### LOCAL

            try:

                print('local')
                print(' ')

                ## main loop
                for k in range(N_max):

                    C,kappa = C_schedule(k),param_schedule(k)

                    #### set x 
                    prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)

                    #### set weights
                    components_ = heval(mat_cvx.value)
                    prev = np.sum(components_/N_train*Q)
                    if mode=='maxmin':
                        Q_hat = Q_hat_mm(components_,param=kappa)
                        Q_star = Q_opt(components_)
                        Q,combis = Q_safe(components_,Q,Q_hat,Q_star,C)
                    elif mode=='softmin':
                        Q_hat = Q_hat_softmin(components_,param=kappa)
                        Q_star = Q_opt(components_)
                        Q,combis = Q_safe(components_,Q,Q_hat,Q_star,C)
                    elif mode=='baratt-boyd':
                        Q,combis = Q_hat_bb(components_,Q,param=kappa),np.array([C])
                    else:
                        Q,combis = Q_opt(components_),np.array([C])
                    w2p(Q)
                    post = np.sum(components_/N_train*Q)
                    F_Q_plus_x_plus = F_param_smc.value
                    G_crit = max(0,prev-post)
                    F_ = Feval(radius.value,centrum.value,mat_cvx.value)
                    print("iter. %04d | Fval. %4.4e | BICval. %4.4e | Gcrit_x. %4.4e | avg. eps. %4.4e " % (k + 1, F_,F_Q_plus_x_plus,G_crit,np.mean(combis)))
                    if (ref_val-F_)<tol_loc and k>10:
                        print('=> convergence to criticality ... polishing phase')
                        Q_final = Q_opt(heval(mat_cvx.value))
                        w2p(Q_final)
                        prob_param_smc.solve(solver=cp.MOSEK,warm_start=True)
                        F_ = Feval(radius.value,centrum.value,mat_cvx.value)
                        print('final::: Fval. %4.4e | BICval. %4.4e' % (F_,prob_param_smc.value))
                        break;
                    else:
                        ref_val = F_Q_plus_x_plus
                if k==N_max-1:
                    F_ = Feval(radius.value,centrum.value,mat_cvx.value)
            except: 
                print('... solver failed | early interruption ')
                F_ = Feval(radius.value,centrum.value,mat_cvx.value)
            print(' ')
            if F_<=f_overallbest:
                print(' /!\ NEW BEST | F = '+str(F_)+'  /!\ ')
                f_overallbest = F_
                mat_out = mat_cvx.value
                center_out = centrum.value
                rad_out = radius.value
                print(' ')
            if k_outer==0:
                endval_single_run.append(F_)
            t.append(time.time()-t_init)
            fval.append(F_)
            repIDs.append(rep)

            ### GLOBAL
            # @check in order not to cycle...
            vals = heval(mat_out)
            active_indices = rho_active_sets(vals,rho=REL_PARAM)
            print('global')
            print(' ')
            try:
                list_cstr_glob_cvx = []
                list_cstr_glob_cvx += [cp.norm(mat_cvx[e]-mat_out[e],'inf')<=R for e in range(B)]
                list_cstr_glob_cvx += [cp.norm(mat_cvx[e]-mat_out[e],'inf')<=R,cp.abs(radius[0]-rad_out[0])<=R]
                ML_loc = bigM_FL(omegas_train,c_centers=mat_cvx.value)
                fun_obj_glob_cvx = h_bar_smc
                t_var = {} 
                for s,omega_s in enumerate(omegas_train):
                    active_indices_s = active_indices[s]
                    l_elected = active_indices_s[0]
                    n_a_s = len(active_indices_s)
                    if n_a_s>1:
                        t_var['t_'+str(s)] = cp.Variable(n_a_s,boolean=True)
                        alpha_s = cp.Variable(1,name='alpha_'+str(s))
                        list_cstr_glob_cvx += [cp.sum(t_var['t_'+str(s)])==1]
                        init_t = np.zeros(n_a_s)
                        init_t[0] += 1
                        t_var['t_'+str(s)].value = init_t
                        fun_obj_glob_cvx += alpha_s[0]
                        alpha_s.value = np.array([weights_train[s]*np.sum(np.abs(omega_s-mat_cvx.value[l_elected]))])
                        for l_plus in active_indices_s:
                            M_bounds = ML_loc[s][l_plus]
                            list_cstr_glob_cvx += [weights_train[s]*cp.norm(omega_s-mat_cvx[l_plus],1)<=alpha_s+cp.sum(cp.multiply(t_var['t_'+str(s)],M_bounds[active_indices_s]))]
                    else:
                        fun_obj_glob_cvx += weights_train[s]*cp.norm(omega_s-mat_cvx[l_elected],1)
                prob_glob_cvx = cp.Problem(cp.Minimize(fun_obj_glob_cvx),list_cstr_glob_cvx+X+symmetry_breaks)
                #local_env.setParam('BestObjStop',f_overallbest*(1-.2*np.sign(f_overallbest)))
                prob_glob_cvx.solve(solver=cp.GUROBI,verbose=True,warm_start=True,env=local_env)
                F_ =  Feval(radius.value,centrum.value,mat_cvx.value)
            except:
                print(' ')
                print('GUROBI failure or manual STOP...')
                F_ = np.inf

            if F_ <= f_overallbest-tol_glob:
                print(' ')
                print(' /!\ NEW BEST | F = '+str(F_)+'  /!\ ')
                print(' ')
                f_overallbest = F_
                t.append(time.time()-t_init)
                fval.append(F_)
                repIDs.append(rep)
            else:
                print(' ')
                print('no improvement after globalization technique... ')
                LO = prob_glob_cvx.status=='optimal'
                break;
        print(' ')
        print('=> '+str(mode)+' (END)')
        print(' ')
        locopt.append(LO)
        endval.append(f_overallbest)
        
    return t,fval,repIDs,locopt,endval,endval_single_run

In [13]:
t_sm,fval_sm,repIDs_sm,locopt_sm,endval_sm,endval_single_run_sm = RRAM(mode='softmin')

=> softmin (START)
 
new rep #1
 
OUTER #1
 
local
 
iter. 0001 | Fval. 1.7752e+00 | BICval. 1.8040e+00 | Gcrit_x. 7.9839e-05 | avg. eps. 6.7526e-01 
iter. 0002 | Fval. 9.9519e-01 | BICval. 1.2972e+00 | Gcrit_x. 1.7410e-03 | avg. eps. 5.3202e-01 
iter. 0003 | Fval. 9.5344e-01 | BICval. 1.0854e+00 | Gcrit_x. 8.7083e-04 | avg. eps. 3.1872e-01 
iter. 0004 | Fval. 7.7465e-01 | BICval. 8.6967e-01 | Gcrit_x. 7.7005e-04 | avg. eps. 3.5466e-01 
iter. 0005 | Fval. 7.7777e-01 | BICval. 8.1419e-01 | Gcrit_x. 2.9852e-04 | avg. eps. 1.6109e-01 
iter. 0006 | Fval. 7.7090e-01 | BICval. 7.8608e-01 | Gcrit_x. 1.3421e-04 | avg. eps. 8.5735e-02 
iter. 0007 | Fval. 7.6527e-01 | BICval. 7.7212e-01 | Gcrit_x. 6.4499e-05 | avg. eps. 6.0851e-02 
iter. 0008 | Fval. 7.5191e-01 | BICval. 7.5681e-01 | Gcrit_x. 5.2195e-05 | avg. eps. 7.3158e-02 
iter. 0009 | Fval. 7.3721e-01 | BICval. 7.4233e-01 | Gcrit_x. 5.4202e-05 | avg. eps. 8.0919e-02 
iter. 0010 | Fval. 6.7724e-01 | BICval. 6.8588e-01 | Gcrit_x. 2.4025e-04 |

ok for 41.53005464480874%
ok for 51.91256830601093%
ok for 62.295081967213115%
ok for 72.6775956284153%
ok for 83.06010928961749%
ok for 93.44262295081968%
                                     CVXPY                                     
                                     v1.3.0                                    
(CVXPY) Jul 25 02:56:30 PM: Your problem has 41 variables, 69 constraints, and 0 parameters.
(CVXPY) Jul 25 02:56:30 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 25 02:56:30 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 25 02:56:30 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY)

-------------------------------------------------------------------------------
(CVXPY) Jul 25 02:56:33 PM: Compiling problem (target solver=GUROBI).
(CVXPY) Jul 25 02:56:33 PM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing -> GUROBI
(CVXPY) Jul 25 02:56:33 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jul 25 02:56:33 PM: Applying reduction Qp2SymbolicQp
(CVXPY) Jul 25 02:56:33 PM: Applying reduction QpMatrixStuffing
(CVXPY) Jul 25 02:56:34 PM: Applying reduction GUROBI
(CVXPY) Jul 25 02:56:34 PM: Finished problem compilation (took 4.214e-01 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Jul 25 02:56:34 PM: Invoking solver GUROBI  to obtain a solution.
Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Changed value of parameter QCPDual

       0      handle free variables                          0s
     113    6.0808793e-01   0.000000e+00   0.000000e+00      0s

Solved in 113 iterations and 0.01 seconds
Optimal objective  6.080879302e-01
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
(CVXPY) Jul 25 02:56:36 PM: Problem status: optimal
(CVXPY) Jul 25 02:56:36 PM: Optimal value: 6.081e-01
(CVXPY) Jul 25 02:56:36 PM: Compilation took 3.483e-01 seconds
(CVXPY) Jul 25 02:56:36 PM: Solver (including time spent in interface) took 5.755e-02 seconds
 
no improvement after globalization technique... 
 
=> softmin (END)
 
new rep #4
 
OUTER #1
 
local
 
iter. 0001 | Fval. 1.5224e+00 | BICval. 1.6790e+00 | Gcrit_x. 7.5783e-04 | avg. eps. 7.4836e-01 
iter. 0002 | Fval. 8.7145e-01 | BICval. 1.1355e+00 | Gcrit_x. 1.6507e-03 | avg. eps. 5.9683

iter. 0010 | Fval. 6.4175e-01 | BICval. 6.4190e-01 | Gcrit_x. 1.7002e-06 | avg. eps. 2.3125e-01 
iter. 0011 | Fval. 6.4175e-01 | BICval. 6.4180e-01 | Gcrit_x. 5.7376e-07 | avg. eps. 2.3727e-01 
iter. 0012 | Fval. 6.4175e-01 | BICval. 6.4176e-01 | Gcrit_x. 1.8835e-07 | avg. eps. 2.5009e-01 
iter. 0013 | Fval. 6.4175e-01 | BICval. 6.4175e-01 | Gcrit_x. 6.1093e-08 | avg. eps. 2.8030e-01 
iter. 0014 | Fval. 6.1538e-01 | BICval. 6.1681e-01 | Gcrit_x. 6.5538e-05 | avg. eps. 2.6121e-01 
iter. 0015 | Fval. 6.1360e-01 | BICval. 6.1430e-01 | Gcrit_x. 1.0015e-05 | avg. eps. 2.2987e-01 
iter. 0016 | Fval. 6.1129e-01 | BICval. 6.1185e-01 | Gcrit_x. 8.5922e-06 | avg. eps. 2.3062e-01 
iter. 0017 | Fval. 6.0867e-01 | BICval. 6.0925e-01 | Gcrit_x. 9.1650e-06 | avg. eps. 2.1430e-01 
iter. 0018 | Fval. 6.0859e-01 | BICval. 6.0876e-01 | Gcrit_x. 2.3941e-06 | avg. eps. 1.3927e-01 
iter. 0019 | Fval. 6.0836e-01 | BICval. 6.0845e-01 | Gcrit_x. 1.3338e-06 | avg. eps. 1.2685e-01 
iter. 0020 | Fval. 6.0832e-01 

In [14]:
endval_sm = np.array(endval_sm)
endval_single_run_sm = np.array(endval_single_run_sm)

In [15]:
np.mean((endval_single_run_sm-endval_sm)/np.abs(endval_single_run_sm))

1.1027789725847926e-05

In [16]:
endval_single_run_sm-endval_sm

array([3.32760757e-05, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00])

In [17]:
max((endval_single_run_sm-endval_sm)/np.abs(endval_single_run_sm))

5.5138948629239634e-05

In [18]:
sum(locopt_sm)

5